In [3]:
from urllib import request 
import itertools
import logging
import random
from pathlib import Path

import numpy as np
try:
    if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
        print("notebook")
        from tqdm import tqdm_notebook as tqdm
    else:
        raise RuntimeError
except (NameError, RuntimeError):
    from tqdm import tqdm
import re
import MeCab

notebook


In [4]:
mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")

In [5]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]

In [6]:
def tokenizer(text, pos=False):
    text = re.sub(r"http://.*\.com", "", text)
    text = re.sub(r"【.*?】", "",text)
    text = text.lower()
    l = [line.split("\t") for line in mecab.parse(text).split("\n")]
    res = [i[0] if not pos else (i[0],i[3]) for i in l 
                   if len(i) >=4 
                       and ("名詞" in i[3] or "動詞" in i[3] or "形容詞" in i[3] )
                       and "数" not in i[3] and "助動詞" not in i[3] and "助詞" not in [3]
                       and  not re.search("[0-9]", i[0])
                       and i[0] not in stopwords
            ]
    return res

In [7]:
tokenizer("認めたくないものだな。自分自身の若さ故の過ちというものを。")

['認め', '自分自身', '若さ故の過ち']

In [8]:
doc_path = "./text/"
doc_dir = Path(doc_path)
dirs = [i for i in doc_dir.iterdir() if i.is_dir()]
dirs

[PosixPath('text/movie-enter'),
 PosixPath('text/it-life-hack'),
 PosixPath('text/kaden-channel'),
 PosixPath('text/topic-news'),
 PosixPath('text/livedoor-homme'),
 PosixPath('text/peachy'),
 PosixPath('text/sports-watch'),
 PosixPath('text/dokujo-tsushin'),
 PosixPath('text/smax')]

In [9]:
articles = [a for categ in dirs for a in categ.iterdir()]

In [10]:
len(articles)

7376

In [11]:
articles = articles[:50]

In [12]:
def read_doc(doc_id):
    with articles[doc_id].open() as f:
        print(f.read())

In [13]:
class Doc_manager():
    def __init__(self, docs):
        self.docs = docs
        
    def read_doc(self, doc_id):
        with self.docs[doc_id].open() as f:
            print(f.read())

In [14]:
dm = Doc_manager(articles)

In [15]:
dm.read_doc(0)

http://news.livedoor.com/article/detail/5978741/
2011-10-30T10:15:00+0900
【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か
　2005年11月から翌2006年7月まで読売新聞にて連載された、直木賞作家・角田光代による初の長編サスペンス『八日目の蝉』。2010年に檀れいと北乃きいの出演によりテレビドラマ化された同作が、2011年4月に永作博美と井上真央の出演によって映画化。そして、劇場公開から半年が過ぎた10月28日、DVD＆ブルーレイとなって発売されました。

八日目の蝉
　妻子ある男と愛し合い、その子を身ごもりながら、あきらめざるをえなかった女。彼女は同時に、男の妻が子供を産んだことを知る。その赤ん坊を見に行った女は、突発的にその子を連れ去り、逃避行を続けた挙句、小豆島に落ち着き、母と娘として暮らしはじめる。


不倫相手の子供を誘拐し、4年間育てた女
　永作博美が演じる野々宮希和子は、不倫相手の子を宿しながらも、彼の「いずれ妻と別れるから、それまで待ってくれ」という常套句を信じて、中絶。後遺症により、二度と子供を産めない身体となってしまいます。その後、不倫相手から彼の妻が出産したことを知らされ、別れを決意。最後に諦めをつけるため、彼らの生後6ヶ月の赤ん坊・恵理菜の顔を見た希和子でしたが、自分に笑顔で向けた恵理菜を見て、思わず誘拐。名前を変えて恵理菜を薫と名付けると、人目を避けて各地を転々とし、二人で幸せな時間を過ごしますが、辿り着いた最後の場所・小豆島で4年の逃避行に終止符を打ちます。


誘拐犯に育てられた女
　4歳になって実の両親と再会を果たした後も、世間から言われの無い中傷を受け、本当の両親との関係を築けないまま、21歳の大学生へと成長した恵理菜。過去と向き合うことを避けてきた恵理菜でしたが、劇団ひとりが演じる不倫相手・岸田孝史の子を宿し、ずっと憎み続けてきた希和子と同じ道を歩んでいることに気付いた彼女は、小池栄子が演じるルポライター・安藤千草と共に、4年間の逃亡生活を追憶する旅に出ます。希和子との幸せだった時間に触れながら、最終地・小豆島に辿り着いた恵理菜が見た真実とは？


八日目の蝉は幸せなのだろうか？
　蝉は俗説として、一生の大半を幼虫として地下で費やし

In [16]:
docs = []
for a in tqdm(articles):
    with a.open() as f:
        f.readline()
        f.readline()
        docs.append(tokenizer(f.read()))

HBox(children=(IntProgress(value=0, max=50), HTML(value='')))

In [17]:
vocab = set(w for d in docs for w in d)
word2id = dict(zip(vocab, itertools.count()))
id2word = dict(list(enumerate(vocab)))

In [18]:
corpus = []
num_words = 0 
for d in docs:
    num_words += len(d)
    corpus.append([word2id[w] for w in d])

In [19]:
random.shuffle(corpus)

In [20]:
num_words

14765

In [21]:
id2word[0]

'good'

In [22]:
len(corpus)

50

In [32]:
test_corpus = corpus[:20]
train_corpus = corpus[20:]

In [33]:
M = len(train_corpus)
#M = len(corpus)
V =  len(vocab)
k = 8

In [34]:
print(M,V,k)

30 4994 8


In [35]:
gamma_ = np.random.rand(M, k)
lambda_ = np.random.rand(V, k)
q_ = np.random.rand(M,V,k)

In [36]:
print(gamma_.shape)
print(lambda_.shape)
print(q_.shape)

(30, 8)
(4994, 8)
(30, 4994, 8)


In [37]:
from scipy.special import digamma

In [38]:
def get_per(corpus, alpha, eta, n_itr=100):
    perplexity = 0.0
    N = 0
    for d in corpus:
        N += len(d)
    for _ in range(n_itr):
        theta = np.array([np.random.dirichlet(a) for a in alpha])
        beta = np.array([np.random.dirichlet(e) for e in eta.T])
        m = np.inner(theta, beta.T)
        log_p = 0.0
        for i, d in enumerate(corpus):
            log_p += np.log(m[i][d]).sum()
        perplexity += np.exp(-log_p/N)
    perplexity = perplexity/n_itr
    return perplexity

In [39]:
alpha = gamma_
eta = lambda_
get_per(test_corpus, alpha, eta, n_itr=200)

7533.3591627705446

In [ ]:
for itr in tqdm(range(100)):
    
    for d in range(M):
        doc = train_corpus[d]
        N_d = len(doc)
        for n in range(N_d):
            gamma_sum = gamma_.sum(axis=1)
            lambda_sum = lambda_.sum(axis=0)
            w = int(doc[n])

            q_[d,w] = np.exp(digamma(gamma_[d]) - digamma(gamma_sum)[d] + digamma(lambda_[w]) - digamma(lambda_sum))
            q_ = q_/q_.sum()
            gamma_[d] += q_.sum(axis=1)[d]
            lambda_[w] += q_.sum(axis=0)[w]
            
            #for i in range(k):
            #    gamma_di = gamma_[d, i]
            #    lambda_wi = lambda_[wl, i]
             #   print(digamma(gamma_sum).shape)
             #   q_[d,w,i] = np.exp(digamma(gamma_di) - digamma(gamma_sum)[d] + digamma(lambda_wi) - digamma(lambda_sum)[i])
             #   gamma_[d,i] += q_.sum(axis=1)[d,i]
             #   lambda_[w,i] += q_.sum(axis=0)[w,i]
                
    alpha = gamma_ - q_.sum(axis=1)
    eta = lambda_ - q_.sum(axis=0)
    
    perplexity = get_per(test_corpus, alpha, eta, n_itr=200)
    print(itr, ": ", perplexity)


HBox(children=(IntProgress(value=0), HTML(value='')))

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/tamoto.yoshifumi/anaconda2/envs/py36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/tamoto.yoshifumi/anaconda2/envs/py36/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/tamoto.yoshifumi/anaconda2/envs/py36/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



0 :  6287.26917582
1 :  6074.75949955
2 :  5955.33258383
3 :  5872.23197668
4 :  5799.02512798
5 :  5707.68683936
6 :  5633.09988448
7 :  5545.13460617
8 :  5465.60170856
9 :  5402.40116459
10 :  5311.30989983
11 :  5224.97043116
12 :  5146.70921357
13 :  5070.61974821
14 :  4981.45065034


In [ ]:
k=4
14 :  5338.01566202
    
k=5
20 :  4814.45180145
    
k=6
20 :  4735.64121761
    
k=7    
20 :  4694.68933233
    


In [ ]:
np.log(p_w.T[test_corpus[0]].T[0].prod())

In [ ]:
p_w.T[test_corpus[0]].T[0].shape

In [ ]:
for i in range(k):
    print("# ", i)
    for t in sorted([(i,rate) for (i,rate) in enumerate(beta.T[i])], key=lambda t: t[1],reverse=True)[:10]:
        print(dictionary[t[0]]," : ",t[1])
    print("\n")

In [ ]:
theta = np.array([np.random.dirichlet(a) for a in alpha])
beta = np.array([np.random.dirichlet(e) for e in eta])

In [ ]:
theta.shape

In [ ]:
theta[0]

In [ ]:
beta[0]

In [ ]:
beta

In [ ]:
p_w = np.inner(theta, beta)

In [ ]:
p_w.shape

In [ ]:
p_w[0].argmax()

In [ ]:
id2word[3156]

In [ ]:
np.log(np.inner(theta, beta)).shape

In [ ]:
def normliation(q):
    return q/q.sum(axis=0)

In [ ]:
q_theta = normliation(np.array([np.random.dirichlet(g) for g in gamma_]).prod(axis=0))
q_beta = normliation(np.array([np.random.dirichlet(l) for l in lambda_]).prod(axis=1))
q_z = q_[d,w,i].prod()

In [ ]:
q_z

In [ ]:
q_theta.shape

In [ ]:
q_beta.shape

In [ ]:
theta = np.array([np.random.dirichlet(g) for g in gamma_])

In [ ]:
theta

In [ ]:
from scipy import stats


def d_KL(p1,p2):
    return stats.entropy(p1, p2, 2)

In [ ]:
theta = np.array([[1/2,1/2],[1/2,1/2]])
beta=  np.array([[1/3,1/3,1/3],[1/3,1/3,1/3]]).T
test_corpus = [[1,2],[0,1]]

In [ ]:
p_w = np.inner(theta, beta)
print(p_w)
log_p = 0.0
N = 0
for i, d in enumerate(test_corpus):
    log_p += np.log(p_w.T[d].T[i].prod())
    N += len(d)
perplexity = np.exp(-log_p/N)

In [ ]:
perplexity